In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--jars lib/graphframes-release-0-5-0-assembly-0.5.0-spark2.1.jar pyspark-shell')
from pyspark.sql import SparkSession
from lib.graphframes import *

spark = SparkSession \
    .builder \
    .appName("RelaTree") \
    .getOrCreate()

# group_members_data = spark.read.csv("data/group_members.csv", header="true").drop("update_time", "is_admin")
# group_channel_data = spark.read.csv("data/group_channel.csv", header="true")

In [4]:
# Vertex DataFrame
v = spark.createDataFrame([
    ("a", "Alice", 34),
    ("b", "Bob", 36),
    ("c", "Charlie", 30),
    ("d", "David", 29),
    ("e", "Esther", 32),
    ("f", "Fanny", 36),
    ("g", "Gabby", 60)
], ["id", "name", "age"])
# Edge DataFrame
e = spark.createDataFrame([
    ("a", "b", "friend"),
    ("b", "c", "follow"),
    ("c", "b", "follow"),
    ("f", "c", "follow"),
    ("e", "f", "follow"),
    ("e", "d", "friend"),
    ("d", "a", "friend"),
    ("a", "e", "friend")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)
g.vertices.show()
g.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+



In [6]:
g.vertices.write.parquet('store/gv.parquet')
g.edges.write.parquet('store/ge.parquet')

agv = spark.read.parquet('store/gv.parquet')
age = spark.read.parquet('store/ge.parquet')

ag = GraphFrame(agv, age)
ag.vertices.show()
ag.edges.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  c|Charlie| 30|
|  e| Esther| 32|
|  a|  Alice| 34|
|  d|  David| 29|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
|  b|    Bob| 36|
+---+-------+---+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+

